In [22]:
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import matplotlib.pyplot
import seaborn as sns
# Too see full dataframe...
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from sklearn.cluster import KMeans
from pandas.io.json import json_normalize

import folium
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.5
  latest version: 4.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.5
  latest version: 4.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [48]:
!pip install xlwt
!pip3 install xlwt
!pip3 install html5lib
# Link To Extract
path='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# Read File
df_wiki=pd.read_html(path)
#Check the type
type(df_wiki)
# Call the position where the table is stored
neighborhood=df_wiki[0]
# Rename the Columns
neighborhood.rename(columns={0:'Postal Code', 1: 'Borough', 2: 'Neighbourhood'}, inplace=True)
# Eliminate the first row
neighborhood=neighborhood.drop([0])
# Eliminate "Not assigned", categorical values from "Borough" Column
neighborhood=neighborhood[neighborhood.Borough !='Not assigned']
# Making DataFrame
neighborhood=pd.DataFrame(neighborhood)
# Merging rows with same Postal Code
neighborhood.set_index(['Postal Code','Borough'],inplace=True)
merge_result = neighborhood.groupby(level=['Postal Code','Borough'], sort=False).agg( ','.join)
# Setting the index
serial_wise=merge_result.reset_index()
# Assign the 'Borough' column value to 'Neighborhood' where 'Not assigned' occurs
serial_wise.loc[4, 'Neighbourhood']='Queen\'s Park'
# Saving the file for future use!
serial_wise.to_excel('wikipedia_table.xls')
# Showing the Data Frame
df=pd.DataFrame(serial_wise)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,Queen's Park


In [49]:
# Geographical Coordinates
df1=pd.read_csv("Geospatial_Coordinates.csv")
#Cancatenation
frames=[df,df1]
frames=pd.concat(frames, axis=1, sort=False)
# Merging the two columns on 'Postal Code'
merge_columns=pd.merge(df, df1, left_on='Postal Code', right_on='Postal Code')
# Save the Data Frame
merge_columns.to_csv('neighbours_geographical.csv')
merge_columns.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [50]:
# Sorting
# set index for only Downtown Toronto
downtown_toronto_data = merge_columns[merge_columns['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
# eliminate 'Postcode' column
downtown_toronto_data=downtown_toronto_data.drop(['Postal Code'], axis=1)
downtown_toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


**Now we will shift our attention towards New York Boroughs. We will select "Manhattan" as a Borough and analyze its neighborhoods later**

In [28]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
nyc_neighborhood_data = newyork_data['features']
nyc_neighborhood_data[2]

{'type': 'Feature',
 'id': 'nyu_2451_34572.3',
 'geometry': {'type': 'Point',
  'coordinates': [-73.82780644716412, 40.887555677350775]},
 'geometry_name': 'geom',
 'properties': {'name': 'Eastchester',
  'stacked': 1,
  'annoline1': 'Eastchester',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.82780644716412,
   40.887555677350775,
   -73.82780644716412,
   40.887555677350775]}}

In [29]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

nyc_neighborhoods = pd.DataFrame(columns=column_names)

for data in nyc_neighborhood_data:
    borough = nyc_neighborhood_data = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    nyc_neighborhoods = nyc_neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
nyc_neighborhoods.head(5)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [30]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(nyc_neighborhoods['Borough'].unique()),
        nyc_neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [31]:
# Creating new Dataframe manhattan_data
manhattan_data = nyc_neighborhoods[nyc_neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


**Using Foursquare API**

In [32]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'RXWR4OET0RZCAP3AG1UF5NYAYCANHNBKZUTXVT3202UJBA3A' # your Foursquare ID
CLIENT_SECRET = 'Z3MTNGFXMF2LVIJXIN3MXWAOEQYD1L5SJZOGMJZ2V2VOGM15' # your Foursquare Secret
VERSION = '20180604'
limit = 20
print('Your credentails:')
print('CLIENT_ID:'+ CLIENT_ID)
print('CLIENT_SECRET:'+ CLIENT_SECRET)

Your credentails:
CLIENT_ID:RXWR4OET0RZCAP3AG1UF5NYAYCANHNBKZUTXVT3202UJBA3A
CLIENT_SECRET:Z3MTNGFXMF2LVIJXIN3MXWAOEQYD1L5SJZOGMJZ2V2VOGM15


In [37]:
# get the geographical coordinates of Downtown Toronto
address = 'Downtown Toronto, ON, Canada'

geolocator = Nominatim(user_agent='Noah')
location = geolocator.geocode(address)
latitude_downtown_toronto = location.latitude
longitude_downtown_toronto = location.longitude
print("Downtown Toronto","latitude",latitude_downtown_toronto, "& " "longitude" ,longitude_downtown_toronto)

Downtown Toronto latitude 43.6563221 & longitude -79.3809161


In [38]:
# Let's get the geographical coordinates of Manhattan.
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent='Noah')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


**Data Visualization**

**(Before the Clustering)**

**Downtown Toronto**

In [51]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

In [52]:
from folium import plugins
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude_downtown_toronto,longitude_downtown_toronto], zoom_start=11)
# instantiate a mark cluster object for the incidents in the dataframe
incidents = plugins.MarkerCluster().add_to(map_downtown_toronto)
# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(incidents)  
    
map_downtown_toronto

**Manhattan**

In [41]:
# let's visualizat Manhattan the neighborhoods in it.
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [42]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

grouping = plugins.MarkerCluster().add_to(map_manhattan)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(grouping)  
    
map_manhattan

**Data Analysis**

We analyze both boroughs/neighborhoods through one hot encoding (giving ‘1’ if a venue category is there, and ‘0’ in case of venue category is not there). On the basis of one hot encoding, we calculate the mean of the frequency of occurrences of each category and then we picked the top ten venues on that basis for each neighborhood. It means that those top venues are showing the most foot traffic or are the most visited places.

**Exploring Neighborhoods in Downtown Toronto**

In [68]:
# Let's create a function to repeat the process to all the neighborhoods in Toronto
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [69]:
# Write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.
downtown_toronto_venues = getNearbyVenues(names=downtown_toronto_data['Neighbourhood'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude'],
                                  )

Regent Park, Harbourfront
Queen's Park
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [70]:
# Let's check the size of the resulting dataframe
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head()

(356, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [71]:
# Let's check how many venues were returned for each neighborhood
downtown_toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,20,20,20,20,20,20
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,20,20,20,20,20,20
Christie,16,16,16,16,16,16
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",20,20,20,20,20,20
"First Canadian Place, Underground city",20,20,20,20,20,20
"Garden District, Ryerson",20,20,20,20,20,20
"Harbourfront East, Union Station, Toronto Islands",20,20,20,20,20,20


In [72]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 125 uniques categories.


**Analyze Each Toronto Neighborhood**

In [73]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_toronto_onehot['Neighbourhood'] = downtown_toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Distribution Center,Electronics Store,Farmers Market,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Gastropub,General Entertainment,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Lake,Liquor Store,Lounge,Martial Arts School,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Museum,Music Venue,Neighborhood,Nightclub,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Pub,Ramen Restaurant,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [74]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Neighbourhood').mean().reset_index()

In [75]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in downtown_toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = downtown_toronto_grouped[downtown_toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0      Farmers Market  0.10
1  Seafood Restaurant  0.10
2         Coffee Shop  0.10
3            Beer Bar  0.10
4            Fountain  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2           Airport  0.06
3               Bar  0.06
4  Sculpture Garden  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.20
1  Italian Restaurant  0.10
2    Sushi Restaurant  0.05
3    Ramen Restaurant  0.05
4                Park  0.05


----Christie----
                venue  freq
0       Grocery Store  0.25
1                Café  0.19
2                Park  0.12
3          Baby Store  0.06
4  Italian Restaurant  0.06


----Church and Wellesley----
            venue  freq
0      Restaurant  0.05
1     Pizza Place  0.05
2  Breakfast Spot  0.05
3       Bookstore  0.05
4   

In [76]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [77]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = downtown_toronto_grouped['Neighbourhood']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Farmers Market,Coffee Shop,Seafood Restaurant,Beer Bar,Cocktail Bar,Bakery,Fountain,Vegetarian / Vegan Restaurant,Thai Restaurant,Museum
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Airport Terminal,Bar,Boat or Ferry,Boutique
2,Central Bay Street,Coffee Shop,Italian Restaurant,Gastropub,Art Museum,Seafood Restaurant,Japanese Restaurant,Spa,Ramen Restaurant,Sushi Restaurant,Sandwich Place
3,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop,Candy Store,Nightclub,Italian Restaurant,Restaurant,Athletics & Sports
4,Church and Wellesley,Café,Salon / Barbershop,Martial Arts School,Beer Bar,Mexican Restaurant,Restaurant,Breakfast Spot,Juice Bar,Creperie,Dance Studio
5,"Commerce Court, Victoria Hotel",Café,Gastropub,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Gym / Fitness Center,American Restaurant,Hotel,Beer Bar
6,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Steakhouse,Gym,Gym / Fitness Center,Pizza Place,Pub,Seafood Restaurant,Bakery
7,"Garden District, Ryerson",Café,Clothing Store,Burger Joint,Comic Shop,College Rec Center,Coffee Shop,Ramen Restaurant,Sporting Goods Shop,Diner,Electronics Store
8,"Harbourfront East, Union Station, Toronto Islands",Plaza,Park,Café,Hotel,Japanese Restaurant,Sporting Goods Shop,Dessert Shop,Supermarket,Salad Place,Ice Cream Shop
9,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Cheese Shop,Farmers Market,Organic Grocery,Grocery Store,Caribbean Restaurant,Mexican Restaurant,Beer Bar,Dessert Shop


**Clustering Toronto Neighborhoods**

In [78]:
# set number of clusters
kclusters = 5

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 3, 1, 2, 2, 1, 1, 1], dtype=int32)

In [79]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
downtown_toronto_merged = downtown_toronto_data

# add clustering labels
downtown_toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Breakfast Spot,Farmers Market,Chocolate Shop,Pub,Restaurant,Performing Arts Venue,Dessert Shop,Bakery
1,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Yoga Studio,Sushi Restaurant,Portuguese Restaurant,Mexican Restaurant,Beer Bar,Creperie,Bank,Chinese Restaurant,Diner
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Café,Clothing Store,Burger Joint,Comic Shop,College Rec Center,Coffee Shop,Ramen Restaurant,Sporting Goods Shop,Diner,Electronics Store
3,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Gastropub,Coffee Shop,Restaurant,Japanese Restaurant,Café,Food Truck,Creperie,Cosmetics Shop,BBQ Joint,Italian Restaurant
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Farmers Market,Coffee Shop,Seafood Restaurant,Beer Bar,Cocktail Bar,Bakery,Fountain,Vegetarian / Vegan Restaurant,Thai Restaurant,Museum


In [80]:
# create map
map_clusters = folium.Map(location=[latitude_downtown_toronto, longitude_downtown_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Neighbourhood'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Examine Each Cluster**

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

**Cluster 1 (Airport Lounge)**

In [81]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Airport Terminal,Bar,Boat or Ferry,Boutique


**Cluster 2 (Coffee and Parks)**

In [82]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 1, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Park,Breakfast Spot,Farmers Market,Chocolate Shop,Pub,Restaurant,Performing Arts Venue,Dessert Shop,Bakery
1,Queen's Park,Coffee Shop,Yoga Studio,Sushi Restaurant,Portuguese Restaurant,Mexican Restaurant,Beer Bar,Creperie,Bank,Chinese Restaurant,Diner
4,Berczy Park,Farmers Market,Coffee Shop,Seafood Restaurant,Beer Bar,Cocktail Bar,Bakery,Fountain,Vegetarian / Vegan Restaurant,Thai Restaurant,Museum
7,"Richmond, Adelaide, King",Pizza Place,Sushi Restaurant,Concert Hall,Lounge,Seafood Restaurant,Neighborhood,Smoke Shop,Plaza,Coffee Shop,Speakeasy
8,"Harbourfront East, Union Station, Toronto Islands",Plaza,Park,Café,Hotel,Japanese Restaurant,Sporting Goods Shop,Dessert Shop,Supermarket,Salad Place,Ice Cream Shop
9,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Café,Restaurant,Japanese Restaurant,Gastropub,Bakery,Gym,Gym / Fitness Center,Train Station,Beer Bar
12,"Kensington Market, Chinatown, Grange Park",Café,Vietnamese Restaurant,Cheese Shop,Farmers Market,Organic Grocery,Grocery Store,Caribbean Restaurant,Mexican Restaurant,Beer Bar,Dessert Shop
16,"St. James Town, Cabbagetown",Restaurant,Café,General Entertainment,Butcher,Deli / Bodega,Diner,Jewelry Store,Bakery,Pub,Japanese Restaurant
18,Church and Wellesley,Café,Salon / Barbershop,Martial Arts School,Beer Bar,Mexican Restaurant,Restaurant,Breakfast Spot,Juice Bar,Creperie,Dance Studio


**Cluster 3 (Coffee, Bars and Restaurants)**

In [83]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 2, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Bay Street,Coffee Shop,Italian Restaurant,Gastropub,Art Museum,Seafood Restaurant,Japanese Restaurant,Spa,Ramen Restaurant,Sushi Restaurant,Sandwich Place
6,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop,Candy Store,Nightclub,Italian Restaurant,Restaurant,Athletics & Sports
14,Rosedale,Park,Playground,Trail,Deli / Bodega,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center
15,Stn A PO Boxes,Beer Bar,Farmers Market,Cocktail Bar,Seafood Restaurant,Restaurant,Park,Jazz Club,Japanese Restaurant,Concert Hall,Tailor Shop
17,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Steakhouse,Gym,Gym / Fitness Center,Pizza Place,Pub,Seafood Restaurant,Bakery


**Cluster 4 (Gastropubs)**

In [84]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 3, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,St. James Town,Gastropub,Coffee Shop,Restaurant,Japanese Restaurant,Café,Food Truck,Creperie,Cosmetics Shop,BBQ Joint,Italian Restaurant


**Cluster 5 (Cafe)**

In [85]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 4, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Garden District, Ryerson",Café,Clothing Store,Burger Joint,Comic Shop,College Rec Center,Coffee Shop,Ramen Restaurant,Sporting Goods Shop,Diner,Electronics Store
10,"Commerce Court, Victoria Hotel",Café,Gastropub,Coffee Shop,Restaurant,Japanese Restaurant,Gym,Gym / Fitness Center,American Restaurant,Hotel,Beer Bar
11,"University of Toronto, Harbord",Café,Bakery,Japanese Restaurant,Bookstore,French Restaurant,Italian Restaurant,Dessert Shop,Comfort Food Restaurant,College Gym,Restaurant


**Exploring Neighbourhoods in Manhattan**

In [86]:
# Let's create a function to repeat the same process to all the neighborhoods in Manhattan
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude', 
                      'Neighborhood Longitude', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
    
    return(nearby_venues)

In [87]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'],
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [88]:
# Let's check how many venues were returned for each neighborhood
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,20,20,20,20,20,20
Carnegie Hill,20,20,20,20,20,20
Central Harlem,20,20,20,20,20,20
Chelsea,20,20,20,20,20,20
Chinatown,20,20,20,20,20,20
Civic Center,20,20,20,20,20,20
Clinton,20,20,20,20,20,20
East Harlem,20,20,20,20,20,20
East Village,20,20,20,20,20,20


In [89]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 211 uniques categories.


**Analyzing Manhattan Neighborhoods**

In [90]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Duty-free Shop,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Heliport,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,New American Restaurant,Noodle House,Opera House,Outdoor Sculpture,Outdoors & Recreation,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pub,Public Art,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,

In [91]:
# Set Index
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()

In [92]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0  Memorial Site  0.15
1           Park  0.15
2     Food Court  0.10
3     Auditorium  0.05
4  Shopping Mall  0.05


----Carnegie Hill----
                  venue  freq
0           Coffee Shop  0.10
1  Gym / Fitness Center  0.10
2    Italian Restaurant  0.10
3                   Gym  0.10
4            Shoe Store  0.05


----Central Harlem----
                venue  freq
0   French Restaurant  0.10
1  African Restaurant  0.10
2           Jazz Club  0.05
3        Cocktail Bar  0.05
4            Boutique  0.05


----Chelsea----
                venue  freq
0             Theater  0.10
1   French Restaurant  0.05
2    Tapas Restaurant  0.05
3         Fish Market  0.05
4  Chinese Restaurant  0.05


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.15
1      Sandwich Place  0.10
2              Museum  0.05
3        Noodle House  0.05
4    Greek Restaurant  0.05


----Civic Center----
                 venue  freq
0            

In [93]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [94]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Memorial Site,Food Court,Auditorium,Sandwich Place,Building,Shopping Mall,Coffee Shop,Gym,Scenic Lookout
1,Carnegie Hill,Coffee Shop,Gym / Fitness Center,Gym,Italian Restaurant,Pizza Place,Café,Dance Studio,French Restaurant,Community Center,Shoe Store
2,Central Harlem,African Restaurant,French Restaurant,Music Venue,Ethiopian Restaurant,Café,Bar,Juice Bar,Bagel Shop,Boutique,Jazz Club
3,Chelsea,Theater,Speakeasy,Hotel,Coffee Shop,New American Restaurant,Chinese Restaurant,Café,Fish Market,French Restaurant,Sushi Restaurant
4,Chinatown,Chinese Restaurant,Sandwich Place,Greek Restaurant,Noodle House,Bubble Tea Shop,Museum,Bike Shop,Salon / Barbershop,New American Restaurant,Cocktail Bar
5,Civic Center,Spa,Yoga Studio,Gym,Cuban Restaurant,Dance Studio,Falafel Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Burrito Place,Bubble Tea Shop
6,Clinton,Gym / Fitness Center,Theater,Hotel,Pizza Place,Peruvian Restaurant,Park,Comedy Club,Coffee Shop,Movie Theater,Café
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Cuban Restaurant,Steakhouse,French Restaurant,Donut Shop,Taco Place,New American Restaurant,Beer Bar
8,East Village,Korean Restaurant,Vietnamese Restaurant,Pet Café,Beer Store,Dog Run,Beer Bar,Bar,Moroccan Restaurant,Bagel Shop,Scandinavian Restaurant
9,Financial District,Coffee Shop,Gym / Fitness Center,Salad Place,Event Space,Pizza Place,Steakhouse,Café,New American Restaurant,Doctor's Office,Falafel Restaurant


**Clustering Manhattan Neighborhoods**

Now we will apply a Machine Learning Technique known as “Clustering” to segment the neighborhoods in similar objects clusters. This will help to analyze the neighborhoods from the tourist's perspective and thereby we can easily extract the tourist places/attractions within our clusters.

**Manhattan**

In [95]:
# Run k-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 0, 0, 4, 0, 0, 1, 0, 4], dtype=int32)

In [96]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Coffee Shop,Gym,Sandwich Place,Discount Store,Supplement Shop,Ice Cream Shop,Pizza Place,Diner,Tennis Stadium,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,4,Chinese Restaurant,Sandwich Place,Greek Restaurant,Noodle House,Bubble Tea Shop,Museum,Bike Shop,Salon / Barbershop,New American Restaurant,Cocktail Bar
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Wine Shop,Park,Breakfast Spot,Italian Restaurant,Market,New American Restaurant,Coffee Shop,Restaurant,Ramen Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Bakery,Deli / Bodega,Park,Yoga Studio,Pet Store,Diner,Café,Farmers Market,Mexican Restaurant,Bistro
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Yoga Studio,Mexican Restaurant,Cocktail Bar,Bakery,Smoke Shop,Café,Mediterranean Restaurant,Caribbean Restaurant,Coffee Shop,Burger Joint


In [97]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Examine Each Cluster**

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

**Manhattan**

**Cluster 1 (Residential Bars, Gyms, Parks and Coffee Shops)**

In [98]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Gym,Sandwich Place,Discount Store,Supplement Shop,Ice Cream Shop,Pizza Place,Diner,Tennis Stadium,Seafood Restaurant
2,Washington Heights,Café,Wine Shop,Park,Breakfast Spot,Italian Restaurant,Market,New American Restaurant,Coffee Shop,Restaurant,Ramen Restaurant
3,Inwood,Bakery,Deli / Bodega,Park,Yoga Studio,Pet Store,Diner,Café,Farmers Market,Mexican Restaurant,Bistro
5,Manhattanville,Coffee Shop,Italian Restaurant,Café,BBQ Joint,Bike Trail,Gastropub,Dumpling Restaurant,Lounge,Supermarket,Bar
6,Central Harlem,African Restaurant,French Restaurant,Music Venue,Ethiopian Restaurant,Café,Bar,Juice Bar,Bagel Shop,Boutique,Jazz Club
8,Upper East Side,Hotel,Bakery,Pizza Place,Cosmetics Shop,Hotel Bar,Park,Coffee Shop,Sandwich Place,Chocolate Shop,Burrito Place
11,Roosevelt Island,Deli / Bodega,Gym,Sandwich Place,Food & Drink Shop,Soccer Field,Outdoors & Recreation,Restaurant,Greek Restaurant,Residential Building (Apartment / Condo),School
12,Upper West Side,American Restaurant,Italian Restaurant,Bakery,Tiki Bar,Southern / Soul Food Restaurant,Chinese Restaurant,Bookstore,Movie Theater,Juice Bar,Greek Restaurant
13,Lincoln Square,Theater,Indie Movie Theater,Concert Hall,Performing Arts Venue,Gym / Fitness Center,College Arts Building,Circus,Opera House,Fountain,Library
14,Clinton,Gym / Fitness Center,Theater,Hotel,Pizza Place,Peruvian Restaurant,Park,Comedy Club,Coffee Shop,Movie Theater,Café


**Cluster 2 (Restaurants and Gyms)**

In [99]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Mexican Restaurant,Thai Restaurant,Bakery,Cuban Restaurant,Steakhouse,French Restaurant,Donut Shop,Taco Place,New American Restaurant,Beer Bar
10,Lenox Hill,Gym,Thai Restaurant,Gift Shop,Middle Eastern Restaurant,Smoke Shop,Liquor Store,Taco Place,Salad Place,Dessert Shop,Gourmet Shop
16,Murray Hill,Japanese Restaurant,Burger Joint,Coffee Shop,Hotel,Shanghai Restaurant,Museum,Sushi Restaurant,Sandwich Place,Tea Room,Grocery Store
29,Financial District,Coffee Shop,Gym / Fitness Center,Salad Place,Event Space,Pizza Place,Steakhouse,Café,New American Restaurant,Doctor's Office,Falafel Restaurant


**Cluster 3 (Theater)**

In [100]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Chelsea,Theater,Speakeasy,Hotel,Coffee Shop,New American Restaurant,Chinese Restaurant,Café,Fish Market,French Restaurant,Sushi Restaurant


**Cluster 4 (Restaurants and Bars)**

In [101]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,West Village,Italian Restaurant,Cocktail Bar,Coffee Shop,Gourmet Shop,Austrian Restaurant,Speakeasy,Candy Store,Mediterranean Restaurant,Boutique,Board Shop


**Cluster 5 (Restaurants, Bars and Books)**

In [102]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Sandwich Place,Greek Restaurant,Noodle House,Bubble Tea Shop,Museum,Bike Shop,Salon / Barbershop,New American Restaurant,Cocktail Bar
4,Hamilton Heights,Yoga Studio,Mexican Restaurant,Cocktail Bar,Bakery,Smoke Shop,Café,Mediterranean Restaurant,Caribbean Restaurant,Coffee Shop,Burger Joint
9,Yorkville,Wine Shop,Deli / Bodega,Italian Restaurant,Gym,Park,Sushi Restaurant,Dog Run,Liquor Store,Bagel Shop,Coffee Shop
18,Greenwich Village,Italian Restaurant,French Restaurant,Café,Sushi Restaurant,Yoga Studio,Coffee Shop,Snack Place,Caribbean Restaurant,Gourmet Shop,Beer Bar
19,East Village,Korean Restaurant,Vietnamese Restaurant,Pet Café,Beer Store,Dog Run,Beer Bar,Bar,Moroccan Restaurant,Bagel Shop,Scandinavian Restaurant
22,Little Italy,Coffee Shop,Wine Bar,Ice Cream Shop,Café,Sandwich Place,Thai Restaurant,Spanish Restaurant,Chinese Restaurant,Salon / Barbershop,Salad Place
26,Morningside Heights,Bookstore,Park,American Restaurant,Greek Restaurant,Outdoor Sculpture,Mexican Restaurant,Farmers Market,Café,Food Truck,Sandwich Place
27,Gramercy,Pizza Place,Coffee Shop,Comedy Club,Spa,Liquor Store,Sushi Restaurant,Gourmet Shop,Beer Bar,Cocktail Bar,Mexican Restaurant
31,Noho,Wine Shop,Rock Club,French Restaurant,Hotel,Gym,Deli / Bodega,Coffee Shop,Cocktail Bar,Sandwich Place,Southern / Soul Food Restaurant
38,Flatiron,Japanese Restaurant,Furniture / Home Store,Cycle Studio,Salon / Barbershop,Salad Place,Russian Restaurant,Miscellaneous Shop,Donut Shop,Gym,Gym / Fitness Center
